In [ ]:
!pip install sqlalchemy


import pandas
import sqlite3

In [ ]:
%load_ext sql


In [ ]:
con = sqlite3.connect("FinalDB")
cur = con.cursor()

df1 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df1.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df2 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df2.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df3 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df3.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")


566

In [ ]:
%sql sqlite:///FinalDB.db


In [ ]:
## Show table columns names
query = 'PRAGMA table_info("CENSUS_DATA");'
result_df = pandas.read_sql(query, con)
print(result_df)

In [16]:
##### 1 - Find the total number of crimes recorded in the CRIME table.
query = 'SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;'
result_df = pandas.read_sql(query, con)
print(result_df)

   COUNT(*)
0       533


In [17]:
##### 2 - List community area names and numbers with per capita income less than 11000.
query = 'SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;'
result_df = pandas.read_sql(query, con)
print(result_df)

  COMMUNITY_AREA_NAME  COMMUNITY_AREA_NUMBER
0  West Garfield Park                   26.0
1      South Lawndale                   30.0
2         Fuller Park                   37.0
3           Riverdale                   54.0


In [19]:
##### 3 - List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)
query = 'SELECT CASE_NUMBER FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE "%MINOR%";'
result_df = pandas.read_sql(query, con)
print(result_df)

  CASE_NUMBER
0    HL266884
1    HK238408


In [21]:
##### 4 - List all kidnapping crimes involving a child?
query = 'SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = "KIDNAPPING" AND DESCRIPTION LIKE "%CHILD%";'
result_df = pandas.read_sql(query, con)
print(result_df)

        ID CASE_NUMBER        DATE                 BLOCK  IUCR PRIMARY_TYPE  \
0  5276766    HN144152  2007-01-26  050XX W VAN BUREN ST  1792   KIDNAPPING   

                DESCRIPTION LOCATION_DESCRIPTION  ARREST  DOMESTIC  ...  \
0  CHILD ABDUCTION/STRANGER               STREET       0         0  ...   

   DISTRICT  WARD  COMMUNITY_AREA_NUMBER  FBICODE X_COORDINATE  Y_COORDINATE  \
0        15  29.0                   25.0       20    1143050.0     1897546.0   

   YEAR   LATITUDE  LONGITUDE                       LOCATION  
0  2007  41.874908 -87.750249  (41.874908413, -87.750249307)  

[1 rows x 21 columns]


In [23]:
##### 5 - List the kind of crimes that were recorded at schools. (No repetitions)
query = 'SELECT DISTINCT PRIMARY_TYPE FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE "%SCHOOL%";'
result_df = pandas.read_sql(query, con)
print(result_df)

             PRIMARY_TYPE
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


In [27]:
##### 6 - List the type of schools along with the average safety score for each type.
query = 'SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS average_safety_score FROM CHICAGO_PUBLIC_SCHOOLS_DATA GROUP BY "Elementary, Middle, or High School";'
result_df = pandas.read_sql(query, con)
print(result_df)

  Elementary, Middle, or High School  average_safety_score
0                                 ES             49.520384
1                                 HS             49.623529
2                                 MS             48.000000


In [31]:
##### 7 - List 5 community areas with highest % of households below poverty line
query = 'SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;'
result_df = pandas.read_sql(query, con)
print(result_df)

  COMMUNITY_AREA_NAME  PERCENT_HOUSEHOLDS_BELOW_POVERTY
0           Riverdale                              56.5
1         Fuller Park                              51.2
2           Englewood                              46.6
3      North Lawndale                              43.1
4  East Garfield Park                              42.4


   cid                                          name     type  notnull  \
0    0                         COMMUNITY_AREA_NUMBER     REAL        0   
1    1                           COMMUNITY_AREA_NAME     TEXT        0   
2    2                    PERCENT_OF_HOUSING_CROWDED     REAL        0   
3    3              PERCENT_HOUSEHOLDS_BELOW_POVERTY     REAL        0   
4    4                   PERCENT_AGED_16__UNEMPLOYED     REAL        0   
5    5  PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA     REAL        0   
6    6              PERCENT_AGED_UNDER_18_OR_OVER_64     REAL        0   
7    7                             PER_CAPITA_INCOME  INTEGER        0   
8    8                                HARDSHIP_INDEX     REAL        0   

  dflt_value  pk  
0       None   0  
1       None   0  
2       None   0  
3       None   0  
4       None   0  
5       None   0  
6       None   0  
7       None   0  
8       None   0  


In [12]:
##### 8 - Which community area is most crime prone? Display the coumminty area number only.
query = 'SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS frequency FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY frequency DESC LIMIT 1;'
result_df = pandas.read_sql(query, con)
print(result_df)

   COMMUNITY_AREA_NUMBER  frequency
0                   25.0         43


In [13]:
##### 9 - Use a sub-query to find the name of the community area with highest hardship index
query = 'SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);'
result_df = pandas.read_sql(query, con)
print(result_df)

  COMMUNITY_AREA_NAME
0           Riverdale


In [14]:
##### 10 - Use a sub-query to determine the Community Area Name with most number of crimes?
query = 'SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1);'
result_df = pandas.read_sql(query, con)
print(result_df)

  COMMUNITY_AREA_NAME
0              Austin
